In [1]:
! pip install -r requirements.txt

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.9.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached astral-3.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached auto_mix_prep-0.2.0-py3-none-any.whl
  Using cached tensorflow-2.13.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.4 kB)
  Using cached pydub-0.25.1-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-24.3.25-py2.py3-none-any.whl.metadata (850 bytes)
  Using cached gast-0.4.0-py3-none-any.whl.metadata (1.1 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached keras-2.13.1-py3-none-any.whl.metadata (2.4 kB)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached numpy-1.24.3-cp310-cp310-ma

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
import os
import glob
import datetime
import pytz
import seaborn as sns
from src.apis import zeroshot_api
import pprint
import datetime
import time
import pytz

warnings.filterwarnings('ignore')

2024-10-26 17:23:29.027263: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-26 17:23:29.751548: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
path =  "./data"

###choose GTT model variant
modelchoice = 'small'
modelpath = f'./GTT/checkpoints/GTT-{modelchoice}'

###whether do 1st-order differencing before prediction, sometimes set autodiff=True will improve prediction accuracy
autodiff = True
PLOT = False
splits = [('2016-02-01', '2016-03-01'), ('2016-03-01', '2016-04-01'), ('2016-04-01', '2016-05-01'),
          ('2016-05-01', '2016-06-01'), ('2016-06-01', '2016-07-01'), ('2016-07-01', '2016-08-01'),
          ('2016-08-01', '2016-09-01'), ('2016-09-01', '2016-10-01'), ('2016-10-01', '2016-11-01')]
experiments = [(96,24), (192,24), (288,24)]

In [11]:
dfCompl = pd.read_csv(f"{path}/train.csv", parse_dates=["date"], index_col=None)
for exp in experiments:
    maes = []
    rmses = []
    mapes = []

    ###define prediction length, i.e., number of forecast steps
    ###define prediction start step, it's N to last step in your data
    pred_len = pred_start =  exp[1]
    data_len = exp[0]
    #if i >= 2:
    #    data_len = 3 * pred_len

    for split in splits:
        dfSplit = dfCompl.iloc[(dfCompl["date"] > pd.to_datetime(split[0]).tz_localize(pytz.timezone('US/Eastern'))).values].reset_index(drop=True)
        dfSplit = dfCompl.iloc[(dfCompl["date"] < pd.to_datetime(split[1]).tz_localize(pytz.timezone('US/Eastern'))).values].reset_index(drop=True)
        
    
        df = dfSplit.iloc[range(len(dfSplit)-(data_len),len(dfSplit))]
        df = df.reset_index(drop=True)
    
        timefeat = df.columns[0]
        targets = [df.columns[1]]
        covariates = df.columns[range(2,len(df.columns))].array

        res = zeroshot_api.forecast(df, targets, covariates, timefeat, pred_len, pred_start, modelpath, autodiff=autodiff)
        if PLOT and res['ret'] == 'success':
            zeroshot_api.plot_res(res)

        trueData = np.array(res["targets"][0]["values"][(data_len-pred_len):])
        preds = np.array(res["targets"][0]["preds"])

        MAE = np.mean(np.abs(trueData - preds))
        RMSE = np.sqrt(np.mean((trueData - preds)**2))
        MAPE = np.mean(np.abs((trueData - preds) / trueData)) * 100

        maes.append(MAE)
        rmses.append(RMSE)
        mapes.append(MAPE)

    print(f"Average results with prediction length {pred_len} and context length {data_len - pred_len}")
    print(f"MAE: {np.mean(maes)}")
    print(f"RMSE: {np.mean(rmses)}")
    print(f"MAPE: {np.mean(mapes)}")

Average results with prediction length 1 and context length 95
MAE: 494.72836960632816
RMSE: 494.72836960632816
MAPE: 9.032356368205487
Average results with prediction length 1 and context length 191
MAE: 491.1343894576999
RMSE: 491.1343894576999
MAPE: 9.739039633814455
Average results with prediction length 1 and context length 287
MAE: 518.145866973941
RMSE: 518.145866973941
MAPE: 9.832997301720635
